# Defining (Almost) Custom Operators #

Python lets you add [custom overloads](https://docs.python.org/3/reference/datamodel.html#special-method-names) for its built-in operators, but it doesn’t let you create entirely new operators.

For example, in computer graphics, it is common to perform arithmetic operations on 3D [vectors](https://en.wikipedia.org/wiki/Vector_space), representing positions, movements and directions in space. Commonly found among these operations are 3 different kinds of multiplication. It is natural to overload Python’s “`*`” operator for elementwise multiplication of two vectors, or multiplication of a vector by a scalar, while the “`@`” operator added in version 3.5 was specifically intended for representing the [dot product](https://en.wikipedia.org/wiki/Dot_product). However, this still leaves no suitable operator to use for the [cross product](https://en.wikipedia.org/wiki/Cross_product). The conventional mathematical symbol for the cross product, “`×`”, is not (currently) part of valid Python syntax. One could suggest picking some other Python operator at random, but this raises the surprise factor, as well as the inconvenience of the wrong operator precedence.

So the usual approach is to give up, and define a method with a boring name like “`cross()`”, so that a mathematical expression like “$v_1 \times v_2$” gets translated in Python as “`v1.cross(v2)`”.

But there is a way to do better than this.


To make things more concrete, let us start by defining a very basic `Vector` class, with just enough functionality to illustrate the issues.

Here it is, with a constructor, a representation method, and elementwise multiplication defined via `__mul__()` and `__rmul__()` methods:

In [ ]:
from numbers import \
    Real

class Vector :

    def __init__(self, x, y, z) :
        self.x = x
        self.y = y
        self.z = z
    #end __init__

    def __repr__(self) :
        return "Vector(%s, %s, %s)" % (self.x, self.y, self.z)
    #end __repr__

    def __mul__(this, that) :
        "by-element multiplication."
        if isinstance(that, Vector) :
            result = Vector(this.x * that.x, this.y * that.y, this.z * that.z)
        elif isinstance(that, Real) :
            result = Vector(this.x * that, this.y * that, this.z * that)
        else :
            result = NotImplemented
        #end if
        return result
    #end __mul__
    __rmul__ = __mul__

#end Vector

This is sufficient to create some `Vector` objects and demonstrate that multiplication by a scalar works either way round:

In [ ]:
v1 = Vector(2, 3, 4)
v2 = Vector(5, 4, 3)
print(v1 * v2, v1 * 3, 3 * v1)

Similarly, we can easily define a `__matmul__()` method to overload the “`@`” operator. But while

    v1 * v2

and

    v1 @ v2

are syntactically valid expressions in Python,

    v1 × v2

is not. However, with suitable definitions behind it, this would be valid:

    v1.cross * v2

That at least avoids the parenthesis clutter of a conventional method call. Now, all we have to do is some strategic rearrangement of the whitespace, to turn it into

    v1 .cross* v2

and now we have something that, while maybe not as concise as the genuine mathematical “$\times$” operator symbol, at least gets us pretty close!

So how do we get this to work? Clearly the `v1.cross` part needs to return a special wrapper for the `v1` object, with its own definition of the “`*`” operator as the cross product. And it needs to do this without the benefit of any explicit pair of parentheses indicating a method call. The custom multiplication method on this wrapper can then operate on that and `v2`, and return a regular `Vector` object as its result.

The obvious way to implement the `.cross` member is as a [descriptor](https://docs.python.org/3/reference/datamodel.html#descriptors). It would be nice to simply write the `.cross()` method in the conventional way, then apply some sort of decorator to turn it into the right sort of descriptor. Here is such a decorator:

In [ ]:
def productop(meth) :

    class methodwrap :

        def __init__(self, instance) :
            self.instance = instance
        #end __init__

        def __call__(self, other) :
            return meth(self.instance, other)
        #end __call__

        def __mul__(self, other) :
            return meth(self.instance, other)
        #end __mul__

    #end methodwrap

    class methodprop :

        def __get__(self, instance, owner) :
            return methodwrap(instance)
        #end ___get__

    #end methodprop

#begin productop
    methodwrap.__name__ = meth.__name__
    methodwrap.__doc__ = meth.__doc__
    return methodprop()
#end productop


Notice how the decorator defines two classes: one is the descriptor, which when invoked returns an instance of the other, which is the wrapper. As well as the `__mul__()` method to implement the custom overload for “`*`”, the wrapper class also defines a `__call__()` method to allow the original decorated function to be invoked using conventional method-call syntax.

How would we use it? Here is a new version of the `Vector` class, adding a cross-product method which is subjected to our custom decorator:

In [ ]:
class Vector :

    def __init__(self, x, y, z) :
        self.x = x
        self.y = y
        self.z = z
    #end __init__

    def __repr__(self) :
        return "Vector(%s, %s, %s)" % (self.x, self.y, self.z)
    #end __repr__

    def __mul__(this, that) :
        "by-element multiplication."
        if isinstance(that, Vector) :
            result = Vector(this.x * that.x, this.y * that.y, this.z * that.z)
        elif isinstance(that, Real) :
            result = Vector(this.x * that, this.y * that, this.z * that)
        else :
            result = NotImplemented
        #end if
        return result
    #end __mul__
    __rmul__ = __mul__

    @productop
    def cross(this, that) :
        "cross product."
        if not isinstance(that, Vector) :
            raise TypeError("cross product only defined between two Vectors")
        #end if
        return \
            Vector \
              (
                this.y * that.z - this.z * that.y,
                this.z * that.x - this.x * that.z,
                this.x * that.y - this.y * that.x,
              )
    #end cross

#end Vector

Conventional method-call syntax still works:

In [ ]:
v1 = Vector(2, 3, 4)
v2 = Vector(5, 4, 3)
print(v1.cross(v2))

But now we can also use our special operator-like syntax!

In [ ]:
print(v1 .cross* v2)

I was hunting around for another character to use that would look more like some kind of mathematical operator, yet be acceptable to Python in an identifier, and I found “ꚛ” U+A69B CYRILLIC SMALL LETTER CROSSED O. That lets you write an expression like this:

    v1 .ꚛ* v2

Can you find any others?

Of course, those who actually use such a letter in their written language might be wondering what all the fuss is about ...

This basic technique could be extended to define any number of custom operators, using all the built-in dyadic ones that Python provides. For example, in addition to basic operations on your custom objects such as

    x1 + x2
    x1 - x2
    x1 * x2
    x1 / x2

one could add a whole other set with entirely different meanings, such as

    x1 .op1+ x2
    x1 .op1- x2
    x1 .op1* x2
    x1 .op1/ x2

and then yet another set on top of this, doing even more strange things:

    x1 .op2+ x2
    x1 .op2- x2
    x1 .op2* x2
    x1 .op2/ x2

In short, you could get entirely carried away.

## But Wait, There’s More ##

But there is a problem with the `productop` decorator as defined above: it doesn’t cope quite as gracefully if you need to perform more than one invocation of the custom operator in a chain. For example,

$$v_1 \times v_2 \times v_3$$

needs to be expressed as

    (v1 .cross* v2) .cross* v3

and we are back to needing extra parentheses again. How can we fix this? Clearly we need to be able to write

    v1 .cross* v2 .cross* v3

which would be implicitly parenthesized as

    (v1.cross * v2.cross) * v3

which means our wrapper’s `__mul__()` method needs to recognize when the right-hand operand is also of wrapper type, and return a wrapped `Vector`, rather than a regular `Vector`, in that situation. Here is a reworked `productop` decorator that does that (note the difference in the definition of `__mul__()`):

In [ ]:
def productop(meth) :

    class methodwrap :

        def __init__(self, instance) :
            self.instance = instance
        #end __init__

        def __call__(self, other) :
            return meth(self.instance, other)
        #end __call__

        def __mul__(self, other) :
            if isinstance(other, methodwrap) :
                this = meth(self.instance, other.instance)
                result = methodwrap(this)
            else :
                result = meth(self.instance, other)
            #end if
            return result
        #end __mul__

    #end methodwrap

    class methodprop :

        def __get__(self, instance, owner) :
            return methodwrap(instance)
        #end ___get__

    #end methodprop

#begin productop
    methodwrap.__name__ = meth.__name__
    methodwrap.__doc__ = meth.__doc__
    return methodprop()
#end productop


Here is exactly the same `Vector` class as before, to save you having to go back to re-execute the class definition:

In [ ]:
class Vector :

    def __init__(self, x, y, z) :
        self.x = x
        self.y = y
        self.z = z
    #end __init__

    def __repr__(self) :
        return "Vector(%s, %s, %s)" % (self.x, self.y, self.z)
    #end __repr__

    def __mul__(this, that) :
        "by-element multiplication."
        if isinstance(that, Vector) :
            result = Vector(this.x * that.x, this.y * that.y, this.z * that.z)
        elif isinstance(that, Real) :
            result = Vector(this.x * that, this.y * that, this.z * that)
        else :
            result = NotImplemented
        #end if
        return result
    #end __mul__
    __rmul__ = __mul__

    @productop
    def cross(this, that) :
        "cross product."
        if not isinstance(that, Vector) :
            raise TypeError("cross product only defined between two Vectors")
        #end if
        return \
            Vector \
              (
                this.y * that.z - this.z * that.y,
                this.z * that.x - this.x * that.z,
                this.x * that.y - this.y * that.x,
              )
    #end cross

#end Vector

Check that two-operand cases still work as before:

In [ ]:
v1 = Vector(2, 3, 4)
v2 = Vector(5, 4, 3)
v3 = Vector(9, 8, 7)
print(v1 .cross* v2, (v1 .cross* v2) .cross* v3)

Try our three-operand case without the helper parentheses:

In [ ]:
print(v1 .cross* v2 .cross* v3)

And it works!